<a href="https://colab.research.google.com/github/tomasborrella/TheValley/blob/main/notebooks/mds%2B2/spark01/Ejercicio_resuelto_MapReduce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio resuelto de MapReduce con Python

Notebook por [Tomás Borrella Martín](https://www.linkedin.com/in/tomasborrella/).

### Enlaces de interés
*   [Slides de presentación](https://docs.google.com/presentation/d/1yMDJfOO1xrOKyoodBF4_nI9ozdKim1OpfjFonuWNX00/edit?usp=sharing)

## 1. Datos
Descargamos un archivo que contiene una canción en cada fila (simplificado para el ejemplo).

NOTA: En un notebook, "!" ejecuta comandos del sistema desde dentro del notebook.

In [ ]:
!wget -P /content/data 'https://raw.githubusercontent.com/tomasborrella/TheValley/main/data/spark01/complete_songs_log.txt' 

Mostramos el contenido del archivo

In [ ]:
# Las primeras 5 líneas
! head -n 5 /content/data/complete_songs_log.txt

In [1]:
# Las últimas 5 líneas
! tail -n 5 /content/data/complete_songs_log.txt

tail: cannot open '/content/data/complete_songs_log.txt' for reading: No such file or directory


In [ ]:
# Vemos cuantas líneas tiene en total
! wc -l /content/data/complete_songs_log.txt

## 2. Instalamos la librería que vamos a usar
mrjob es un paquete que permite ejecutar MapReduce jobs con Python

[Documentación oficial](https://mrjob.readthedocs.io/en/latest/index.html#)

In [ ]:
! pip install mrjob

## 3. Creamos el archivo del programa

In [ ]:
%%file wordcount_final.py
# %%file: "magic function" que guarda el contenido de una celda a un archivo

# Importamos la librería mrjob
from mrjob.job import MRJob 

# Creamos una clase que hereda de MRJob
class MRSongCount(MRJob):
    
    # En la fase de Map cada linea del archivo de texto se lee como una pareja 
    # clave valor (key, value) que son los parámetros 2 y 3 del "mapper".
    # En este caso cada linea no contiene ninguna key sino solo el valor, 
    # por eso usamos _ como segundo parámetro (campo key).

    def mapper(self, _, line):
        line_splited = line.split(", ")
        song = line_splited[0]
        # La salida es una dupla (nombre_cancion, 1) 
        yield (song, 1)

    # En la fase de Reduce se combina cada dupla con la misma clave.
    # (en el ejemplo la clave es el nombre de la canción)
    # En este caso la forma de combinarlo es sumando, lo que nos dará
    # el total de reproducciones de cada canción
    
    def reducer(self, key, values):
        # La salida es una dupla (nombre_cancion, suma_de_reproducciones)
        yield (key, sum(values))
        
if __name__ == "__main__":
    MRSongCount.run()

## 4. Ejecutamos el programa

In [ ]:
# run the code as a terminal command
! python wordcount_final.py /content/data/complete_songs_log.txt